In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
from datasets import load_dataset


device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

batch_size = 64
block_size = 128
learning_rate = 3e-5
max_itters = 3000
eval_itters = 100
n_embd = 384
n_head = 8
dropout = 0.2
n_layer = 8
epoch = 50

c:\Users\Sachin\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using cuda device


In [2]:
data = load_dataset("wikitext", "wikitext-2-raw-v1")
train_data = data["train"]['text']
test_data = data["validation"]['text']

train_text = "".join(train_data)
test_text = "".join(test_data)

chars = sorted(list(set(train_text + test_text)))
vocab_size = len(chars)

In [3]:
string_to_int = {c: i for i, c in enumerate(chars)}
int_to_string = {i: c for i, c in enumerate(chars)}
encode = lambda x: [string_to_int[c] for c in x]
decode = lambda x: "".join([int_to_string[i] for i in x])

In [4]:
train_data = torch.tensor(encode(train_text)).to(device)
test_data = torch.tensor(encode(test_text)).to(device)

def get_batch(split):
    data = train_data if split == "train" else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [5]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        v = self.value(x)
        out = wei @ v
        return out

In [6]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [7]:
class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [8]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

In [9]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            *[Block(n_embd, n_head=n_head) for _ in range(n_layer)]
        )
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape

        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            index_cond = index[:, -block_size:]
            logits, loss = self.forward(index_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index


model = GPTLanguageModel(vocab_size).to(device)

In [10]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "test"]:
        losses = torch.zeros(eval_itters)
        for k in range(eval_itters):
            x, y = get_batch(split)
            logits, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean().item()
    model.train()
    return out

In [11]:
# vocab_size = len(chars)
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# for i in range(epoch):
#     with open('lenisha.pkl', 'rb') as f:
#         model = pickle.load(f)
#         print("Model loaded")
#     model.to(device)
#     optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#     for itters in range(max_itters):
#         if itters % eval_itters == 0:
#             losses = estimate_loss()
#             print(
#                 f"Iteration {i * 3000 + itters}, Train loss: {losses['train']:.4f}, Test loss: {losses['test']:.4f}"
#             )

#         x, y = get_batch("train")

#         logits, loss = model.forward(x, y)

#         optimizer.zero_grad(set_to_none=True)
#         loss.backward()
#         optimizer.step()

#     with open('lenisha.pkl', 'wb') as f:
#         pickle.dump(model, f)
#         print("Model saved")
        

# print("Final loss: ", loss.item())

In [12]:
import os
vocab_size = len(chars)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for i in range(epoch):
    if i > 0 and os.path.exists('lenisha.pkl'):
        with open('lenisha.pkl', 'rb') as f:
            model = pickle.load(f)
            print(f"Epoch {i}: Model loaded")
        model.to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    else:
        if i == 0:
            print("Epoch 0: Starting with fresh model")

    for itters in range(max_itters):
        if itters % eval_itters == 0:
            losses = estimate_loss()
            print(f"Epoch {i}, Iteration {i * max_itters + itters}, Train loss: {losses['train']:.4f}, Test loss: {losses['test']:.4f}")
        x, y = get_batch("train")
        logits, loss = model(x, y)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    with open('lenisha.pkl', 'wb') as f:
        pickle.dump(model, f)
        print(f"Epoch {i}: Model saved")

print("Final loss:", loss.item())

Epoch 0: Starting with fresh model
Epoch 0, Iteration 0, Train loss: 7.0463, Test loss: 7.0480
Epoch 0, Iteration 100, Train loss: 3.4989, Test loss: 3.5062
Epoch 0, Iteration 200, Train loss: 2.9142, Test loss: 2.9191
Epoch 0, Iteration 300, Train loss: 2.7119, Test loss: 2.7148
Epoch 0, Iteration 400, Train loss: 2.6100, Test loss: 2.6118
Epoch 0, Iteration 500, Train loss: 2.5574, Test loss: 2.5609
Epoch 0, Iteration 600, Train loss: 2.5224, Test loss: 2.5235
Epoch 0, Iteration 700, Train loss: 2.4972, Test loss: 2.4960
Epoch 0, Iteration 800, Train loss: 2.4731, Test loss: 2.4770
Epoch 0, Iteration 900, Train loss: 2.4496, Test loss: 2.4484
Epoch 0, Iteration 1000, Train loss: 2.4134, Test loss: 2.4083
Epoch 0, Iteration 1100, Train loss: 2.3642, Test loss: 2.3637
Epoch 0, Iteration 1200, Train loss: 2.3159, Test loss: 2.3181
Epoch 0, Iteration 1300, Train loss: 2.2791, Test loss: 2.2780
Epoch 0, Iteration 1400, Train loss: 2.2375, Test loss: 2.2369
Epoch 0, Iteration 1500, Train l

In [1]:
context = torch.tensor(encode("my name is lenisha  "), dtype=torch.long, device=device)
generated_chars = decode(model.generate(context.unsqueeze(0), 500)[0].tolist())
print(generated_chars)

NameError: name 'torch' is not defined

In [16]:
import shutil
shutil.copyfile("lenisha.pkl", "lenisha_backup.pkl")
print("Backed up lenisha.pkl")

Backed up lenisha.pkl


In [ ]:
import pickle
from datasets import load_dataset

data = load_dataset("wikitext", "wikitext-2-raw-v1")
train_data = data["train"]['text']
test_data = data["validation"]['text']
train_text = "".join(train_data)
test_text = "".join(test_data)
chars = sorted(list(set(train_text + test_text)))

with open('lenisha.pkl', 'rb') as f:
    model = pickle.load(f)

with open('lenisha.pkl', 'wb') as f:
    pickle.dump({"model": model, "chars": chars}, f)
    print("Updated lenisha.pkl with model and vocabulary")

with open('lenisha.pkl', 'rb') as f:
    data = pickle.load(f)
    print("Loaded keys:", data.keys())
    print("Number of characters in vocab:", len(data["chars"]))

Updated lenisha.pkl with model and vocabulary
Loaded keys: dict_keys(['model', 'chars'])
Number of characters in vocab: 1118
